In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Here I have Imported Packages and ignored Warnings**

In [ ]:
import pandas as pd, numpy as np, os
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns',None)

import random
import math
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing, model_selection, metrics

In [ ]:
dir = '../input/tabular-playground-series-nov-2021/'
z = '.csv'

train = pd.read_csv(dir+'train'+z)
test = pd.read_csv(dir+'test'+z)

sample_submission = pd.read_csv(dir+'sample_submission'+z)

**Here We observed that dataset contains 600000 rows and 102 columns**

Out of these columns non of the column have null value

In [ ]:
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
train.describe()

**Skewness is present in data**

In [ ]:
train.skew()

Here We have splited out dataset into dependant and independant features

In [ ]:
X_main = train.drop(columns='target')
y_main = train['target']

Here We are finding the chi square test for 50 features which gives us maximum variance.

Chi Square Test to narrow down to 50 features

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
X_norm = MinMaxScaler().fit_transform(X_main)
chi_selector = SelectKBest(chi2, k=50)
chi_selector.fit(X_norm, y_main)
chi_support = chi_selector.get_support()
chi_feature = X_main.loc[:,chi_support].columns.tolist()
print(str(len(chi_feature)), 'selected features')
chi_feature

**Here we have imported the PCA for dimentionality reduction**

1.In PCA we are reducing independant features such that 90% of data is described by output of PCA or eigen vectors.

2.In PCA we observed that 45 features contains 90% of data so we derrived PCA for 45 features as shown in graph

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data_rescaled = scaler.fit_transform(X_main)
pca = PCA(n_components = 0.90)
pca.fit(data_rescaled)
reduced = pca.transform(data_rescaled)

In [ ]:
reduced.shape

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')

In [ ]:
pca = PCA().fit(data_rescaled)

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,6)

fig, ax = plt.subplots()
xi = np.arange(1, 102, step=1)
y = np.cumsum(pca.explained_variance_ratio_)

plt.ylim(0.0,1.1)
plt.plot(xi, y, marker='o', linestyle='--', color='b')

plt.xlabel('Number of Components')
plt.xticks(np.arange(0, 100, step=5)) #change from 0-based array index to 1-based human-readable label
plt.ylabel('Cumulative variance (%)')
plt.title('The number of components needed to explain variance')

plt.axhline(y=0.90, color='r', linestyle='-')
plt.text(0.5, 0.85, '90% cut-off threshold', color = 'red', fontsize=16)

ax.grid(axis='x')
plt.show()

In [ ]:
reduced[0]

**XGB algorithm**

In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split 
x_, x_val_, y_, y_val_ = train_test_split(reduced, y_main, train_size = 0.85, random_state = 0)

**Logistic Algorithm**

In [ ]:

algorithm = LogisticRegression(penalty='l2', dual=False, tol=1e-4,
C=1.0, fit_intercept=True,
intercept_scaling=1, class_weight=None,
random_state=None, solver='lbfgs',
max_iter=100, multi_class='auto',
verbose=0, warm_start=False, n_jobs=None,
l1_ratio=None)
Logistic_Regression_Model = algorithm.fit(x_, y_)

y_pred = Logistic_Regression_Model.predict(x_val_)

print('Logistic Regression AUC:', roc_auc_score(y_val_, y_pred))
        

**ANN**

In [ ]:
x = train.iloc[:, :-1]
y = train['target']
x_test = test.iloc[:,:]
y_test = sample_submission.iloc[:, :]

In [ ]:
x_train = pd.get_dummies(x)
y_train = pd.get_dummies(y)
x_test = pd.get_dummies(x_test)
y_test = pd.get_dummies(y_test)

In [ ]:
missing_val = x.isnull().sum()
print(missing_val[missing_val > 0])
x, x_val, y, y_val = train_test_split(x, y, train_size = 0.85, random_state = 0)

In [ ]:
import keras 
from tensorflow.keras import layers 
from tensorflow.keras.callbacks import EarlyStopping 

model = keras.Sequential([ 
    layers.BatchNormalization(),
    layers.Dense(2048, activation='sigmoid', input_shape=[101]),
    layers.Dropout(rate = 0.3), 
    layers.Dense(2500, activation='sigmoid'),
    layers.Dropout(rate = 0.3),
    layers.BatchNormalization(),    
    layers.Dense(2500, activation='sigmoid'), 
    layers.Dropout(rate = 0.3), 
    layers.BatchNormalization(),
    layers.Dense(2048, activation='sigmoid'), 
    layers.Dropout(rate = 0.3), 
    layers.Dense(1, activation = 'sigmoid'), 
]) 

early_stopping = EarlyStopping(min_delta = 0.001, 
patience = 10, restore_best_weights = True)

In [ ]:
model.compile( 
    optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['AUC'],
    )

In [ ]:
prediction = model.fit( 
    x, y, 
    validation_data=(x_val, y_val), 
    batch_size=256, 
    epochs=50, 
    callbacks = [early_stopping], 
    verbose = 2, 
)

**Random Forest Algorithm**

In [ ]:
y_pred_test = Logistic_Regression_Model.predict(reduced_test)


In [ ]:
print(sample_submission['target'],y_pred_test)

In [ ]:
y_final = model.predict(pd.get_dummies(test))


In [ ]:
y_final.shape

In [ ]:

sample_submission['target'] = y_final

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.to_csv('submission.csv', index=False)